In [ ]:
from google.colab import drive
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Mount Google Drive
drive.mount('/content/drive')

# Define data transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset (Make sure the path is correct and points to your dataset on Google Drive)
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/test2', transform=transform)

# Dataset Info
print("Classes:", dataset.classes)
print("Class-to-Index Mapping:", dataset.class_to_idx)
print("Number of Samples:", len(dataset))

# Initialize the model with dropout
def initialize_model_with_dropout(model_name, num_classes=2, dropout_rate=0.5):
    if model_name == "alexnet":
        model = models.alexnet(pretrained=True)
        # Freeze convolution layers
        for param in model.features.parameters():
            param.requires_grad = False
        # Replace the final fully connected layers with a Dropout layer and a new classifier
        model.classifier[6] = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(model.classifier[6].in_features, num_classes)
        )
    return model

# Calculate metrics function
def calculate_metrics(model, loader, device):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    conf_matrix = confusion_matrix(all_labels, all_predictions)
    TN, FP, FN, TP = conf_matrix.ravel()

    # Accuracy: (TP + TN) / Total
    total = conf_matrix.sum()
    accuracy = (TP + TN) / total if total > 0 else 0.0

    # Precision: TP / (TP + FP)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0

    # Recall: TP / (TP + FN)
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0

    # F1-Score: 2 * (Precision * Recall) / (Precision + Recall)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return accuracy, precision, recall, f1, conf_matrix

# Train the model function with validation accuracy printed after each epoch
def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5):
    best_val_accuracy = 0
    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

        # Validation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%")


        # Track the best model with the highest validation accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy

    return best_val_accuracy

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cross-validation setup
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# DataLoader setup
batch_size = 32
learning_rates = [0.0001, 0.001, 0.01]
optimizers_to_test = {'Adam': optim.Adam, 'SGD': optim.SGD}
results = {model_name: {opt_name: [] for opt_name in optimizers_to_test} for model_name in ["alexnet"]}

# Cross-validation loop
for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f"Fold {fold_idx + 1}/{num_folds}")

    # Create training/validation split
    train_val_data = torch.utils.data.Subset(dataset, train_val_idx)
    test_data = torch.utils.data.Subset(dataset, test_idx)

    # Further split training/validation
    train_size = int(0.8 * len(train_val_data))
    val_size = len(train_val_data) - train_size
    train_data, val_data = torch.utils.data.random_split(
        train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
    )

    # DataLoaders
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    for model_name in ["alexnet"]:
        best_lr_for_optimizer = {}  # Track the best learning rate for each optimizer

        for optimizer_name, optimizer_fn in optimizers_to_test.items():
            best_val_accuracy = 0  # Track best validation accuracy for this optimizer
            best_lr = None  # Track best learning rate for this optimizer

            for lr in learning_rates:
                # Initialize the model with dropout
                model = initialize_model_with_dropout(model_name, dropout_rate=0.5).to(device)
                criterion = nn.CrossEntropyLoss()

                # Choose optimizer based on the current configuration
                if optimizer_name == 'Adam':
                    optimizer = optimizer_fn(model.parameters(), lr=lr)
                else:
                    optimizer = optimizer_fn(model.parameters(), lr=lr, momentum=0.9)

                # Train the model and get validation accuracy
                print(f"Training {model_name} with {optimizer_name} and LR={lr}")
                _ = train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
                # Evaluate on Validation set
                validation_metrics = calculate_metrics(model, val_loader, device)
                val_accuracy = validation_metrics[0]  # Get the validation accuracy

                print(f"Validation Accuracy for {model_name} with {optimizer_name} and LR={lr}: {val_accuracy:.2f}%")

                # Update the best learning rate if needed
                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    best_lr = lr

            # After finding the best learning rate, evaluate on the test set
            print(f"Best learning rate for {optimizer_name} is {best_lr} with validation accuracy {best_val_accuracy:.2f}%")

            # Initialize the model with the best learning rate and train again
            model = initialize_model_with_dropout(model_name, dropout_rate=0.5).to(device)
            criterion = nn.CrossEntropyLoss()

            if optimizer_name == 'Adam':
                optimizer = optim.Adam(model.parameters(), lr=best_lr)
            else:
                optimizer = optim.SGD(model.parameters(), lr=best_lr, momentum=0.9)

            # Train the model with the best learning rate
            _ = train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)

            # Now evaluate on the test set using the best learning rate
            test_metrics = calculate_metrics(model, test_loader, device)
            print(f"Test Metrics for {model_name} with {optimizer_name} and Best LR={best_lr}:")
            print(f"Accuracy={test_metrics[0]:.2f}, Precision={test_metrics[1]:.2f}, Recall={test_metrics[2]:.2f}, F1-Score={test_metrics[3]:.2f}, Confusion Matrix=\n{test_metrics[4]}")

            # Store the results for this fold
            results[model_name][optimizer_name].append({
                'lr': best_lr,
                'accuracy': test_metrics[0],
                'precision': test_metrics[1],
                'recall': test_metrics[2],
                'f1': test_metrics[3],
                'confusion_matrix': test_metrics[4]
            })